In [2]:
#%pip install duckdb 
import pandas as pd
import glob
import time
import duckdb


conn=duckdb.connect()

In [3]:
# with pandas
cur_time = time.time()
df = pd.concat([pd.read_csv(f) for f in glob.glob('C:\\Users\\oshato02\\Documents\\My programming\\My programming\\MyDuck\\archive\\*.csv')])
print(f"time: {(time.time() - cur_time)}")
print(df.head(10))


time: 0.39090394973754883
  Order ID                     Product Quantity Ordered Price Each  \
0   176558        USB-C Charging Cable                2      11.95   
1      NaN                         NaN              NaN        NaN   
2   176559  Bose SoundSport Headphones                1      99.99   
3   176560                Google Phone                1        600   
4   176560            Wired Headphones                1      11.99   
5   176561            Wired Headphones                1      11.99   
6   176562        USB-C Charging Cable                1      11.95   
7   176563  Bose SoundSport Headphones                1      99.99   
8   176564        USB-C Charging Cable                1      11.95   
9   176565          Macbook Pro Laptop                1       1700   

       Order Date                        Purchase Address  
0  04/19/19 08:46            917 1st St, Dallas, TX 75001  
1             NaN                                     NaN  
2  04/07/19 22:30      

In [4]:
# with duckdb
cur_time = time.time()
df = conn.execute("""
	SELECT *
	FROM read_csv_auto('C:\\Users\\oshato02\\Documents\\My programming\\My programming\\MyDuck\\archive\\*.csv', Header=True)
	LIMIT 10
""").df()
print(f"time: {(time.time() - cur_time)}")
print(df)

time: 0.08628392219543457
  Order ID                     Product Quantity Ordered Price Each  \
0   176558        USB-C Charging Cable                2      11.95   
1     None                        None             None       None   
2   176559  Bose SoundSport Headphones                1      99.99   
3   176560                Google Phone                1        600   
4   176560            Wired Headphones                1      11.99   
5   176561            Wired Headphones                1      11.99   
6   176562        USB-C Charging Cable                1      11.95   
7   176563  Bose SoundSport Headphones                1      99.99   
8   176564        USB-C Charging Cable                1      11.95   
9   176565          Macbook Pro Laptop                1       1700   

       Order Date                        Purchase Address  
0  04/19/19 08:46            917 1st St, Dallas, TX 75001  
1            None                                    None  
2  04/07/19 22:30      

In [5]:
df = conn.execute("""
	SELECT *
	FROM read_csv_auto('C:\\Users\\oshato02\\Documents\\My programming\\My programming\\MyDuck\\archive\\*.csv', Header=True)
""").df()
conn.register("df_view", df)
conn.execute("DESCRIBE df_view").df()

,column_name,column_type,null,key,default,extra
0,Order ID,VARCHAR,YES,None,None,NaN
1,Product,VARCHAR,YES,None,None,NaN
2,Quantity Ordered,VARCHAR,YES,None,None,NaN
3,Price Each,VARCHAR,YES,None,None,NaN
4,Order Date,VARCHAR,YES,None,None,NaN
5,Purchase Address,VARCHAR,YES,None,None,NaN


In [6]:
conn.execute("SELECT COUNT(*) FROM df_view").df()

,count_star()
0,186850


In [7]:
df.isnull().sum()
df = df.dropna(how='all')

In [8]:
conn.execute("SELECT * FROM df").df()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
2,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
3,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176561,Wired Headphones,1,11.99,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001"
...,...,...,...,...,...,...
186300,259353,AAA Batteries (4-pack),3,2.99,09/17/19 20:56,"840 Highland St, Los Angeles, CA 90001"
186301,259354,iPhone,1,700,09/01/19 16:00,"216 Dogwood St, San Francisco, CA 94016"
186302,259355,iPhone,1,700,09/23/19 07:39,"220 12th St, San Francisco, CA 94016"
186303,259356,34in Ultrawide Monitor,1,379.99,09/19/19 17:30,"511 Forest St, San Francisco, CA 94016"


In [13]:
conn.execute("""SELECT * FROM df WHERE "Order ID"='295665'""").df()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,295665,Macbook Pro Laptop,1,1700,12/30/19 00:01,"136 Church St, New York City, NY 10001"


In [13]:
conn.execute("""
CREATE OR REPLACE TABLE sales AS
	SELECT
		"Order ID"::INTEGER AS order_id,
		Product AS product,
		"Quantity Ordered"::INTEGER AS quantity,
		"Price Each"::DECIMAL AS price_each,
		strptime("Order Date", '%m/%d/%y %H:%M')::DATE as order_date,
		"Purchase Address" AS purchase_address
	FROM df
	WHERE
		TRY_CAST("Order ID" AS INTEGER) NOTNULL
""")

In [11]:
conn.execute("DESCRIBE sales").df()

,column_name,column_type,null,key,default,extra
0,order_id,INTEGER,YES,NaN,NaN,NaN
1,product,VARCHAR,YES,NaN,NaN,NaN
2,quantity,INTEGER,YES,NaN,NaN,NaN
3,price_each,"DECIMAL(18,3)",YES,NaN,NaN,NaN
4,order_date,DATE,YES,NaN,NaN,NaN
5,purchase_address,VARCHAR,YES,NaN,NaN,NaN


In [14]:
conn.execute("Select * FROM sales").df()

,order_id,product,quantity,price_each,order_date,purchase_address
0,176558,USB-C Charging Cable,2,11.95,2019-04-19,"917 1st St, Dallas, TX 75001"
1,176559,Bose SoundSport Headphones,1,99.99,2019-04-07,"682 Chestnut St, Boston, MA 02215"
2,176560,Google Phone,1,600.00,2019-04-12,"669 Spruce St, Los Angeles, CA 90001"
3,176560,Wired Headphones,1,11.99,2019-04-12,"669 Spruce St, Los Angeles, CA 90001"
4,176561,Wired Headphones,1,11.99,2019-04-30,"333 8th St, Los Angeles, CA 90001"
...,...,...,...,...,...,...
185945,259353,AAA Batteries (4-pack),3,2.99,2019-09-17,"840 Highland St, Los Angeles, CA 90001"
185946,259354,iPhone,1,700.00,2019-09-01,"216 Dogwood St, San Francisco, CA 94016"
185947,259355,iPhone,1,700.00,2019-09-23,"220 12th St, San Francisco, CA 94016"
185948,259356,34in Ultrawide Monitor,1,379.99,2019-09-19,"511 Forest St, San Francisco, CA 94016"


In [34]:
conn.execute("""
	SELECT 
		* EXCLUDE (product, order_date, purchase_address)
	FROM sales
	""").df()

,order_id,quantity,price_each
0,176558,2,11.95
1,176559,1,99.99
2,176560,1,600.00
3,176560,1,11.99
4,176561,1,11.99
...,...,...,...
185945,259353,3,2.99
185946,259354,1,700.00
185947,259355,1,700.00
185948,259356,1,379.99


In [38]:
conn.execute("""
	SELECT 
		MIN(COLUMNS(* EXCLUDE (product, order_date, purchase_address))) 
	FROM sales
	""").df()

,min(sales.order_id),min(sales.quantity),min(sales.price_each)
0,141234,1,2.99


In [46]:
conn.execute("""
	SELECT 
		sales.order_id, sales.quantity, sales.price_each
	FROM sales
    Inner join sales as sales1
             on sales.order_id=sales1.order_id
             and sales1.order_id in (141234,319670)
            
	""").df()

,order_id,quantity,price_each
0,319670,1,99.99
1,141234,1,700.00


In [47]:
conn.execute("""
	CREATE OR REPLACE VIEW aggregated_sales AS
	SELECT
		order_id,
		COUNT(1) as nb_orders,
		MONTH(order_date) as month,
		str_split(purchase_address, ',')[2] AS city,
		SUM(quantity * price_each) AS revenue
	FROM sales
	GROUP BY ALL
""")

In [50]:
conn.execute("From aggregated_sales").df()

,order_id,nb_orders,month,city,revenue
0,196942,1,5,Los Angeles,11.99
1,196943,1,5,Dallas,11.99
2,196949,1,5,San Francisco,5.98
3,196954,1,5,San Francisco,99.99
4,196962,1,5,Austin,14.95
...,...,...,...,...,...
178432,196494,1,5,New York City,14.95
178433,196712,1,5,Boston,14.95
178434,194233,1,5,San Francisco,150.00
178435,196652,1,5,Boston,2.99


In [51]:
conn.execute("""Select 
             city,
             sum(revenue) as total
             From aggregated_sales
             Group by 1
             Order by 2
             """).df()

,city,total
0,Austin,1819581.75
1,Portland,2320490.61
2,Seattle,2747755.48
3,Dallas,2767975.40
4,Atlanta,2795498.58
5,Boston,3661642.01
6,New York City,4664317.43
7,Los Angeles,5452570.80
8,San Francisco,8262203.91


In [53]:
conn.execute("FROM aggregated_sales.parquet").df()

,order_id,nb_orders,month,city,revenue
0,196942,1,5,Los Angeles,11.99
1,196943,1,5,Dallas,11.99
2,196949,1,5,San Francisco,5.98
3,196954,1,5,San Francisco,99.99
4,196962,1,5,Austin,14.95
...,...,...,...,...,...
178432,297444,1,12,New York City,150.00
178433,297463,1,12,Los Angeles,14.95
178434,297468,1,12,Los Angeles,3.84
178435,297486,1,12,San Francisco,14.95


In [52]:
conn.execute("COPY (FROM aggregated_sales) TO 'aggregated_sales.parquet' (FORMAT 'parquet')")